In [ ]:
import requests
import time

headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

# request package를 사용하여 error handling
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        # 한글처럼 아스키가 아닌 문자들을 변환시킬 때 서비스키도 같이 바꿔버림
        resp.raise_for_status()
        #if(resp.headers['Content-Type'].split("/")[0] == "image"):
            #resp.headers['Content-Type'].split("/")[1] #확장자
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(e.response.status_code)
            print(e.response.reason)
            time.sleep(timeout) # 1초 정도 기다림
            print(maxretries)
            print("재시도") #maxretries
            download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome('/Users/eunha/Desktop/2019 빅데이터 청년인재/chromedriver')

In [ ]:
driver.get("https://ticket.interpark.com")

In [ ]:
driver.get_cookies()

In [ ]:
[_["name"] for _ in driver.get_cookies()]

In [ ]:
[_["name"] for _ in driver.get_cookies()]

In [ ]:
import json
account = {"id":"1002galaxy", "pw":"dmskzla12."}

In [ ]:
driver.window_handles

In [ ]:
# 로그인 버튼 찾기
def login(account, method="login"):
    try:
        login = driver.find_element_by_css_selector("#logstatus > a")
        if method == "login":
            # 로그인 버튼을 찾고 로그인되었는지 확인
            if login.get_attribute("id") and login.get_attribute("id") == "aLogin":
                #로그인 안 되어 있다면 로그인!
                print("로그인 진행")
                driver.get(login.get_attribute("href"))
                driver.find_element_by_css_selector("#UID").send_keys(account["id"])
                driver.find_element_by_css_selector("#PWD").send_keys(account["pw"])
                driver.find_element_by_css_selector("div.loginBtn > button").click()
            else:
                print("이미 로그인 되어 있음")
        else:
            # 로그아웃
            print("로그아웃 진행")
            login.click()
    except:
        # 실행 중 에러 뜨면 False return
        return False
    return True

In [ ]:
login(account, "login")
# login(Account, "logout") <- 로그아웃

In [ ]:
_login = [_["name"] for _ in driver.get_cookies()]

In [ ]:
_logout = [_["name"] for _ in driver.get_cookies()]

In [ ]:
set(_login) - set(_logout)

In [ ]:
# 로그인되어 있으면 로그아웃, 로그아웃되어 있으면 로그인
if "ISession%5FID" not in [_["name"] for _ in driver.get_cookies()]:
    login(account)
else:
    login(None, "logout")

In [ ]:
# 공연 관련 목록 뽑아오기
# 기본적으로는 콘서트 주소를 리턴해주지만 내가 원하는 게 있으면 그 주소값을 리턴
def gnbbtns(category="콘서트"):
    links = [_ for _ in driver.find_elements_by_css_selector("ul.gnbBtns a[class^=btn]")]
    titles = [_.find_element_by_css_selector("img").get_attribute("alt") for _ in links]
    if category in titles:
        return links[titles.index(category)].get_attribute("href")
    return False

In [ ]:
url = gnbbtns("콘서트")
if url:
    driver.get(url)

In [ ]:
# 공연 정보 불러오기
# 다음에 좌측 콘서트탭 우측아래 전체보기 버튼 눌러서 해보는 거 해보기..
def hotissue(issue=None):
    issues = [_ for _ in driver.find_elements_by_css_selector("div.wrap_issue dl.list > dt.issue_obj > a")]
    title = [_.find_element_by_css_selector("span.txt1").text for _ in issues]
    if issue in title:
        return issues[title.index(issue)].get_attribute("href")
    else:
        return issues[0].get_attribute("href")
        

In [ ]:
hotissue("박정현")

In [ ]:
driver.get(hotissue("박정현"))

In [ ]:
driver.find_element_by_css_selector("div.tk_dt_btn_TArea a span").click()

In [ ]:
driver.window_handles

In [ ]:
driver.switch_to_window(driver.window_handles[0])

In [ ]:
def bookWindow(options=None):
    try:
        if options:
            print("옵션 있음")
            iframe = driver.find_element_by_name("ifrCalendar")
            print("여기까지")
            if iframe:
                driver.switch_to_frame(iframe)
                dates = [_ for _ in driver.find_elements_by_css_selector("td.able > a")]
                title = [_.text for _ in dates]
            
            if options in title:
                driver.execute_script(dates[title.index(options)].get_attribute("onclick"))
            else:
                # 무조건 앞에 꺼 선택
                driver.execute_script(dates[0].get_attribute("onclick"))
            
            driver.switch_to_default_content()
        driver.find_element_by_css_selector(".tk_dt_btn_TArea > a span").click()
        driver.switch_to_window(driver.window_handles[-1])
        return True
    except:
        return False

In [ ]:
driver.switch_to_window(driver.window_handles[-1])

In [ ]:
bookWindow("20")

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
iframe = driver.find_element_by_name("ifrCalendar")
print(driver.current_url)
datelist = [(_.find_element_by_css_selector("a").text, _.find_element_by_css_selector("a").get_attribute("href"), _.find_element_by_css_selector("a").get_attribute("onclick")) for _ in driver.find_elements_by_css_selector("div.cal_Wrap tr td") if _.get_attribute("id")]
driver.switch_to_window(driver.window_handles[-1])
driver.find_element_by_css_selector(".tk_dt_btn_TArea > a span").click()
datelist

In [ ]:
driver.execute_script(datelist[0][1]+datelist[0][2])

In [ ]:
datelist = [(_.find_element_by_css_selector("a").text for _ in driver.find_elements_by_css_selector("div.cal_Wrap tr td") if _.get_attribute("id")]
datelist

In [ ]:
driver.switch_to_window(driver.window_handles[-1])

In [ ]:
def safeClose():
    if len(driver.window_handles) > 1:
        for _ in driver.window_handles[1:]:
            driver.switch_to_window(_)
            driver.close()
    driver.switch_to_window(driver.window_handles[0])
    driver.switch_to_default_content()

In [ ]:
driver.window_handles

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
driver.find_element_by_id("LargeNextBtnImage").click()

In [ ]:
import re
seat = re.compile('<img src=".+?" id=".+?" class=".+?" style=".+?" onclick="(.+?)" seatinfo=".+?">')

In [ ]:
seatiframe = driver.find_element_by_id("ifrmSeat")
driver.switch_to_frame(seatiframe)
seatdetailiframe = driver.find_element_by_id("ifrmSeatDetail")
driver.switch_to_frame(seatdetailiframe)

In [ ]:
seatlist = seat.findall(driver.page_source)

In [ ]:
seatinfore = re.compile("javascript: SelectSeat\('.+?', '.+?', '.+?', '(.+?)', '(.+?)', '.+?'\)")

In [ ]:
seatinfore.findall(seatlist[1]) #구역, 열, 좌석번호

In [ ]:
seatinfore.findall(seatlist[1])[0][1]

In [ ]:
seatlist[0]

In [ ]:
#좌석 선택 안 되었을 때 제일 앞 좌석
district='Z'
row = 14
col = 14
seatindex = 0
print(seatlist[seatindex])
for _ in seatlist[1:]:
    thisdistrict = seatinfore.findall(_)[0][0]
    thisrow = int(seatinfore.findall(_)[0][1])
    thiscol = int(seatinfore.findall(_)[0][2])
    if thisdistrict < seatinfore.findall(seatlist[seatindex])[0][0]:
#         print(thisdistrict)
        seatindex = seatlist.index(_)
        continue
    elif thisdistrict == seatinfore.findall(seatlist[seatindex])[0][0]:
        if thisrow < int(seatinfore.findall(seatlist[seatindex])[0][1]):
#             print(thisrow)
            seatindex = seatlist.index(_)
            continue
        elif thisrow == int(seatinfore.findall(seatlist[seatindex])[0][1]):
            if thiscol < int(seatinfore.findall(seatlist[seatindex])[0][2]):
#                 print(thisrow)
                seatindex = seatlist.index(_)
                continue
            else:
                continue
        else:
            continue
    else:
        continue
print(seatlist[seatindex])
print(seatindex)

In [ ]:
driver.execute_script(seatlist[0])

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
seatiframe = driver.find_element_by_id("ifrmSeat")
driver.switch_to_frame(seatiframe)

In [ ]:
driver.execute_script("fnSelect()")

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
bookstepiframe = driver.find_element_by_id("ifrmBookStep")
driver.switch_to_frame(bookstepiframe)

In [ ]:
onfocusscript = driver.find_element_by_css_selector("#PriceRow001 td select").get_attribute("onfocus")
onchangescript = driver.find_element_by_css_selector("#PriceRow001 td select").get_attribute("onchange")

In [ ]:
from selenium.webdriver.support.ui import Select

In [ ]:
select = Select(driver.find_element_by_css_selector("#PriceRow001 td select"))

In [ ]:
select.select_by_index(1)

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
driver.execute_script(driver.find_element_by_id("SmallNextBtnLink").get_attribute("href"))

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
bookstepiframe = driver.find_element_by_id("ifrmBookStep")
driver.switch_to_frame(bookstepiframe)

In [ ]:
driver.find_element_by_id("YYMMDD").send_keys("951124")

In [ ]:
driver.switch_to_window(driver.window_handles[1])

In [ ]:
driver.execute_script(driver.find_element_by_id("SmallNextBtnLink").get_attribute("href"))

In [ ]:
safeClose()

In [ ]:
driver.switch_to_window(driver.window_handles[-1])

In [ ]:
driver.close()